## Bracket Challenge Predictor

Reading data

In [220]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

from functools import reduce
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

from xgboost import XGBRegressor

In [321]:
team_matchups = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Tournament Matchups.csv")

away_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Away.csv")
home_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Home.csv")
neutral_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Neutral.csv")

conf_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Conference Stats.csv")
team_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Team Results.csv")
team_v_ranked = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/TeamRankings.csv")

ken_eff = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Efficiency.csv")
ken_height = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Height.csv")
ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")

conf_mapping = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/REF _ NCAAM Conference and ESPN Team Name Mapping.csv")

cur_coaches = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/REF _ Current NCAAM Coaches.csv")
coach_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Coach Results.csv")

seed_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Seed Results.csv")
upset_count = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Count.csv")
upset_inf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Seed Info.csv")

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_28025/3145123047.py:13: DtypeWarning: Columns (144) have mixed types. Specify dtype option on import or set low_memory=False.
  ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")


since ken_ datasets don't have TEAMNO var will first create seperate lookup table to create a column for its respective id

In [322]:
team_ids = team_matchups[['YEAR','TEAM','TEAM NO']].drop_duplicates()
conf_ids = conf_perf[['YEAR','CONF','CONF ID']].drop_duplicates()

team_res.rename(columns={'TEAM ID':'TEAM NO'},inplace=True)

ken_eff.rename(columns={'Season':'YEAR','Team':'TEAM'},inplace=True)
ken_height.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_conf.rename(columns={'Season':'YEAR','Short Conference Name':'CONF'},inplace=True)

ken_eff = ken_eff.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_height = ken_height.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_conf = ken_conf.merge(conf_ids,on=['YEAR','CONF'],how='left')

In [323]:
temp_away = away_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]
temp_home = home_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]
temp_neutral = neutral_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]

#adds _A to all columns to represent Away
temp_away.columns = list(temp_away.columns[:2]) + [col + "_A" for col in temp_away.columns[2:]]

#_H for Home
temp_home.columns = list(temp_home.columns[:2]) + [col + "_H" for col in temp_home.columns[2:]]

#_N for Neutral
temp_neutral.columns = list(temp_neutral.columns[:2]) + [col + "_N" for col in temp_neutral.columns[2:]]


#combined stats as they are highly correlated away, home, and neutral
#function to create new combined dataframe easily
def var_mean(dfs, vars):
    combined_df = [dfs[0].loc[:,vars],dfs[1].loc[:,vars],dfs[2].loc[:,vars]]
    combined_df = reduce(lambda left, right: pd.merge(left, right, on=['YEAR','TEAM NO']),combined_df)
    for var in vars[2:]:
        combined_df[var] = combined_df[[var+'_x',var+'_y',var]].mean(axis=1)
        combined_df.drop([var+'_x',var+'_y'],axis=1,inplace=True)
    return combined_df

vars = ['YEAR','TEAM NO','BADJ EM','FTR','FTRD',
        'BADJ T','TOV%','TOV%D','OREB%','OP OREB%','RAW T','WAB',
        '2PT%', '2PT%D', '3PT%', '3PT%D', 'BLK%', 'BLKED%',
        'AST%', 'OP AST%', '2PTR', '3PTR', '2PTRD', '3PTRD']
dfs = [away_stats,home_stats,neutral_stats]

team_stats = var_mean(dfs,vars)

temp_res = team_res.loc[:,['TEAM','PAKE','PASE','R64','R32','S16','E8','F4',
                           'F2','CHAMP','TOP2']]

temp_v_ranked = team_v_ranked.loc[:,['YEAR','TEAM NO','TR RATING','V 1-25 WINS','V 1-25 LOSS',
                                     'V 26-50 WINS','V 26-50 LOSS','LUCK RATING','CONSISTENCY TR RATING']]

temp_ken_eff = ken_eff.loc[:,['YEAR','TEAM NO','Adjusted Offensive Efficiency','Adjusted Defensive Efficiency',
                              'Avg Possession Length (Offense)','Avg Possession Length (Defense)']]

temp_height = ken_height.loc[:,['YEAR','TEAM NO','AvgHeight','EffectiveHeight','Experience','Bench']]


Creating supplmental dataset that aren't dependent on team metrics (i.e: conference, coach, upsets)

Created a column on who won each matchup for every game in the dataset. Will now add columns of interest to this dataset and will try to do feature engineering as to model who'd win each matchup.

In [324]:
wins = []
for i in range(len(team_matchups)//2):
    if team_matchups.iloc[i,8] > team_matchups.iloc[i+1,8]:
        wins.append(1)
        wins.append(0)
    else:
        wins.append(0)
        wins.append(1)
    i += 2

comp_stats = team_matchups.iloc[:,[0,3,4,5,6,7,8]]
comp_stats['WIN'] = wins
        
comp_stats.head()

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_28025/1098123238.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_stats['WIN'] = wins


,YEAR,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,WIN
0,2024,1067,Connecticut,1,1,64,91,1
1,2024,1026,Stetson,16,64,64,52,0
2,2024,1060,Florida Atlantic,8,64,64,65,0
3,2024,1036,Northwestern,9,32,64,77,1
4,2024,1029,San Diego St.,5,16,64,69,0


In [325]:
dfs = [comp_stats,temp_away,temp_home,temp_neutral,temp_v_ranked,temp_height,temp_ken_eff,team_stats]

comp_team_stats = reduce(lambda left, right: pd.merge(left, right, on=['YEAR','TEAM NO']),dfs)

comp_team_stats = comp_team_stats.merge(temp_res,on='TEAM',how='left')

comp_team_stats.head()

,YEAR,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,WIN,BADJ O_A,BADJ D_A,WIN%_A,EFG%_A,EFG%D_A,PPPO_A,PPPD_A,BADJ O_H,BADJ D_H,WIN%_H,EFG%_H,EFG%D_H,PPPO_H,PPPD_H,BADJ O_N,BADJ D_N,WIN%_N,EFG%_N,EFG%D_N,PPPO_N,PPPD_N,TR RATING,V 1-25 WINS,V 1-25 LOSS,V 26-50 WINS,V 26-50 LOSS,LUCK RATING,CONSISTENCY TR RATING,AvgHeight,EffectiveHeight,Experience,Bench,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),BADJ EM,FTR,FTRD,BADJ T,TOV%,TOV%D,OREB%,OP OREB%,RAW T,WAB,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,PAKE,PASE,R64,R32,S16,E8,F4,F2,CHAMP,TOP2
0,2024,1067,Connecticut,1,1,64,91,1,129.5,101.7,75.000000,56.6,50.6,1.206,1.109,123.7,89.4,100.000000,57.5,41.4,1.261,0.881,126.7,88.5,100.000000,57.0,44.2,1.195,0.954,22.3,10,2,4,0,2.4,10.8,78.42,1.53,2.43106,28.14,127.5,91.1,18.6,17.5,33.433333,32.600000,32.600000,65.433333,14.733333,15.533333,35.200000,26.100000,66.266667,3.766667,58.933333,44.266667,36.033333,31.666667,13.633333,8.233333,65.200000,46.866667,59.400000,40.600000,67.666667,32.333333,10.8,13.3,10,6,5,5,5,4,4,2
1,2024,1026,Stetson,16,64,64,52,0,106.0,115.4,37.500000,49.9,54.9,1.047,1.152,110.0,113.4,84.615385,56.6,50.2,1.171,1.097,120.9,109.0,100.000000,61.5,42.5,1.253,1.028,-2.0,0,1,0,1,2.7,12.7,77.68,1.34,1.78601,22.03,109.8,114.3,18.4,17.5,-0.300000,28.166667,22.600000,65.600000,16.300000,12.566667,29.966667,32.633333,65.500000,-2.280000,55.800000,50.300000,37.566667,31.900000,7.933333,8.800000,52.300000,48.100000,58.066667,41.933333,60.233333,39.766667,0.0,0.0,1,0,0,0,0,0,0,0
2,2024,1060,Florida Atlantic,8,64,64,65,0,114.1,108.2,50.000000,51.7,52.3,1.144,1.111,118.1,101.6,92.857143,56.5,47.2,1.213,0.987,120.9,97.9,77.777778,56.6,50.6,1.147,1.045,12.7,1,1,1,0,-0.1,9.9,76.55,-0.32,2.57785,32.11,117.8,102.1,16.5,18.3,15.133333,35.166667,28.166667,69.066667,15.966667,16.533333,33.166667,29.500000,69.100000,0.400000,55.966667,49.400000,35.466667,34.300000,7.600000,6.800000,50.466667,38.400000,61.433333,38.566667,64.133333,35.866667,2.8,2.7,2,1,1,1,1,0,0,0
3,2024,1036,Northwestern,9,32,64,77,1,112.4,97.9,36.363636,49.5,51.8,1.027,1.088,121.5,103.7,88.235294,56.6,52.3,1.212,1.050,107.6,87.6,50.000000,48.3,48.0,0.999,0.952,11.7,3,5,3,3,1.5,10.6,77.59,0.43,2.51412,24.66,115.9,99.5,18.9,18.1,17.433333,27.400000,39.633333,64.433333,14.400000,18.666667,25.933333,27.233333,64.400000,0.503333,48.166667,49.100000,37.933333,35.400000,10.500000,9.900000,56.700000,56.533333,62.266667,37.733333,62.533333,37.466667,0.9,0.8,3,3,0,0,0,0,0,0
4,2024,1029,San Diego St.,5,16,64,69,0,110.8,96.2,42.857143,48.1,49.4,1.051,1.044,110.1,90.7,92.307692,52.4,46.2,1.095,0.911,119.3,97.3,80.000000,48.0,44.5,1.155,1.023,13.1,2,1,2,5,0.7,10.2,77.56,0.60,2.56787,29.61,113.4,94.0,17.7,18.3,18.666667,40.300000,33.133333,66.700000,14.866667,17.200000,32.633333,28.933333,66.900000,0.866667,50.966667,48.133333,31.566667,29.633333,12.200000,8.533333,52.000000,49.066667,63.200000,36.800000,59.766667,40.233333,0.8,1.1,11,6,4,1,1,1,0,1
